In [31]:
import os
import itertools
from collections import OrderedDict

In [32]:
def loadDataSet(file):
    t = open(file,"r").read().split("\n")
    return t

In [43]:
#file = ['input-data.txt','parameter-file.txt']
#file = ['small-data/data-1.txt','small-data/para1-2.txt']
file = ['data-1.txt','para1-2.txt']
candidates_set = []
frequent_items_set = []
transactions = loadDataSet(file[0])
transactions = [t.strip("{}") for t in transactions]
transactions = [x.split((',')) for x in transactions]
transactions = [[x.strip(" ") for x in t] for t in transactions]
len(transactions)

62

In [44]:
itemMinSupport = {}
with open(file[1],"r") as f:
    for row in f:
        row_val = row.replace("=",":").split(":")
        
        if not row.isspace():
            if row_val[0] != 'SDC ':
                if row_val[0][4].isdigit():
                    row_val[0] = row_val[0][4:].replace(") ","")    
                row_val[1] = row_val[1].strip(" ")
            itemMinSupport[row_val[0]] = row_val[1].strip("\n") 
#print("itemMinSupport: ",itemMinSupport)
# must_have = itemMinSupport.popitem()[1] 
must_have = itemMinSupport.get("must-have")
if must_have:
    del itemMinSupport["must-have"]
    must_have = list(must_have.split(" or "))
else:
    must_have = []
cannot_have = itemMinSupport.get("cannot_be_together")
if cannot_have:
    del itemMinSupport["cannot_be_together"]
    cannot_have = list(cannot_have.split("},"))
    cannot_have = list([y.split(",") for y in [x.strip(" {}") for x in cannot_have]])
    cannot_have = [[x.strip(" ") for x in t] for t in cannot_have]
else:
    cannot_have = []
sdc = float(itemMinSupport.pop("SDC "))
print("must have: %s cannot have: %s sdc: %s" %(must_have, cannot_have, sdc))

must have: ['30', '60'] cannot have: [['30', '60']] sdc: 0.4


In [45]:
itemMinSupport = OrderedDict(sorted(itemMinSupport.items(), key=lambda t: t[1]))
itemMinSupport


OrderedDict([('10', '0.10'),
             ('80', '0.20'),
             ('30', '0.30'),
             ('40', '0.40'),
             ('60', '0.40'),
             ('20', '0.50'),
             ('70', '0.70'),
             ('50', '0.80')])

In [46]:
def must_have_in_itemset(itemset, must_have_list):
    if len(must_have_list) > 0:
        for i in must_have_list:
            if i in itemset:
                return True
    return False


def cannot_together_in_itemset(itemset,cannot_together_list):
    cannot_together_items_count = 0
    if len(cannot_together_list) > 0:
        for cannot_together_itemset in cannot_together_list:
            for i in cannot_together_itemset:
                if i in itemset:
                    cannot_together_items_count = cannot_together_items_count+1
                if cannot_together_items_count >= len(i):
                    return True
    return False

def cannot_be_together_check(freq_item_set,cannot_have):

    if len(cannot_have) > 0:
        for item_set in cannot_have:
            for i in range(len(freq_item_set)):
                #print("item_set: ",item_set)
                #print("freq_item_set at index i %s: %s" %(i,freq_item_set[i]))
                item_index = 0
                if i > 0 and freq_item_set[i]:
                    f_k = [x[:-2] for x in freq_item_set[i]]
                    for j in f_k:
                        #print("f_k: %s, item_index: %s" %(f_k,item_index))
                        if set(item_set) <= set(f_k[item_index]):
                            #print("popitem freq_item_set[i]", freq_item_set[i])
                            freq_item_set[i].remove(freq_item_set[i][item_index])
                            f_k.remove(f_k[item_index])
                            item_index = item_index -1
                        item_index = item_index + 1
            
    return freq_item_set

def must_have_check(freq_item_set, must_have):

    if len(must_have) > 0:
        must_have_present = False
        for i in range(len(freq_item_set)):
            #print ("in must have, fres_item_set: ", freq_item_set[i])
            item_set_index = 0
            for item_set in range(len(freq_item_set[i])):
                if i > 0 and freq_item_set[i]:
                    f_k = [x[:-2] for x in freq_item_set[i]]
                else:
                    f_k = freq_item_set[i]
                #print("in must_have f_k: ", f_k)
                #print("index: ", item_set_index)
                if bool(set(must_have) & set(f_k[item_set_index])) == False:
                    freq_item_set[i].remove(freq_item_set[i][item_set_index])
                    item_set_index = item_set_index - 1
                    #print("adjusted item set index: ", item_set)
                item_set_index += 1
    return freq_item_set


def constraints_check(f_list, must_have_list, cannot_together_list):
    for itemset in f_list:
        # print(must_have_in_itemset(itemset, must_have_list))
        # print(cannot_together_in_itemset(itemset, cannot_together_list))
        
        if (not(must_have_in_itemset(itemset, must_have_list)) or cannot_together_in_itemset(itemset, cannot_together_list)):
            f_list.pop(f_list.index(itemset))
             
    return f_list


def init_pass(transactions, itemMinSupport):
    candidate = {}
    MIS = next (iter (itemMinSupport.values()))
    print (MIS)
    for row in itemMinSupport.items():
        count = 0
        for t in transactions:
            #print("item : %s transaction : %s" %(row[0],t))
            #print(t.count(row[0]))
            count += t.count(row[0])
            
        #print(count)
        if(count/len(transactions)) >= float(MIS):
            candidate.append(row[0])
    return candidate

def init_pass1(transactions, itemMinSupport):
    candidate = []
    l_item = []
    MIS = next (iter (itemMinSupport.values()))
    for row in itemMinSupport.items():
        count = 0
        support = 0
        
        for t in transactions:
            #print("item : %s transaction : %s" %(row[0],t))
            #print(t.count(row[0]))
            count += t.count(row[0])
        support = (count/len(transactions))
        #print("count: ", count)
        #print("support: ", support)
        if support >= float(MIS):
            l_item = [row[0], float(row[1]), support]
            candidate.append(l_item)
    return candidate


    

In [47]:
l = init_pass1(transactions, itemMinSupport)
l

[['10', 0.1, 0.4032258064516129],
 ['80', 0.2, 0.2903225806451613],
 ['30', 0.3, 0.4838709677419355],
 ['40', 0.4, 0.41935483870967744],
 ['60', 0.4, 0.7419354838709677],
 ['20', 0.5, 0.4838709677419355],
 ['70', 0.7, 0.6612903225806451],
 ['50', 0.8, 0.9354838709677419]]

In [48]:
frequent_items_set.append(list([x[0].split(',') for x in l if x[2] >= x[1]]))
print(frequent_items_set)
#frequent_items_set = cannot_be_together_check(frequent_items_set,cannot_have)


[[['10'], ['80'], ['30'], ['40'], ['60'], ['50']]]


In [49]:
def findSubsets(itemSet,subsetLength):
    print("itemSet:",itemSet," subsetLength:",subsetLength)
    print(list(set(itertools.combinations(itemSet, subsetLength))))
    return list(set(itertools.combinations(itemSet, subsetLength)))


def level_2_candidate_generation(l,sdc):
    candidate_list = []
    l_index = 0
    for l_items in l:
        l_index += 1
        if l_items[2] >= l_items[1] :
            for h_items in l[l_index:]:
                if h_items[2] >= l_items[1] and abs(h_items[2] - l_items[2]) <= sdc:
                    candidate_list.append(l_items[0]+","+h_items[0])
                    #candidate_list.append([l_items[0],h_items[0]])
        
    return candidate_list

def MScandidate_gen(frequent_items,sdc): 
    candidate_gen = []
    index_f = 0
    frequent_item_pair = False
    for f1 in range(len(frequent_items)):
        k_items = len(frequent_items[f1])
        item_1 = frequent_items[f1]
        print("item_1",item_1)
        for f2 in range(f1+1,len(frequent_items)):
            item_2 = frequent_items[f2]
            print("item_2",item_2)
            cand = []
            for i in range(k_items-1):
                if item_1[i] == item_2[i]:
                    frequent_item_pair = True
                else:
                    frequent_item_pair = False
                    break
            if frequent_item_pair == True:
                print("frequent_items_set:  sup cal",frequent_items)
                sup_f1 = [x[2] for x in l if item_1[k_items-1] == x[0]]
                print("sup_f1: ",sup_f1)
                sup_f2 = [x[2] for x in l if item_2[k_items-1] == x[0]]
                print("sup_f2: ",sup_f2)
                print("frequent_items_set:  after sup cal",frequent_items)
                print("f1[k_items-1] : %s itemMinSupport.get(f1[k_items-1]) : %s"%(frequent_items[f1][k_items-1],itemMinSupport.get(frequent_items[f1][k_items-1])))
                print("f2[k_items-1] : %s itemMinSupport.get(f2[k_items-1]) : %s"%(frequent_items[f2][k_items-1],itemMinSupport.get(frequent_items[f2][k_items-1])))
                if (itemMinSupport.get(item_1[k_items-1]) < itemMinSupport.get(item_2[k_items-1])) and (abs(float(sup_f2[0]) - float(sup_f1[0])) <= sdc):
                    print("frequent_items_set: before c",frequent_items)
                    cand = item_1 + list(str(item_2[k_items-1]).split(','))
                    print("frequent_items_set: after c",frequent_items)
                    #cand_item=frequent_items[f2][k_items-1] 
                   # print("cand_item", cand_item)
                    #cand.append(cand_item)
                    print("frequent_items_set: after append",frequent_items)
                    #c = list(set(f1 + f2))
                    candidate_gen.append(item_1 + list(str(item_2[k_items-1]).split(',')))
                    print("candidate: ",candidate_gen)
                    print("frequent_items_set:",frequent_items)
                if len(cand) > 1: 
                    subsets = findSubsets(cand, len(cand)-1)
                    print("subsets: ", subsets)
                    for s in subsets:
                        print("list s: %s, c[0]: %s "%(list(s),cand[0]))
                       #print("itemMinSupport.get(c[1]): %s, itemMinSupport.get(c[0]): %s" %(itemMinSupport.get(c[1]) , itemMinSupport.get(c[0])))
                        print("frequent_items_set: ",frequent_items)
                        
                        if (cand[0] in list(s)) or (itemMinSupport.get(cand[1]) == itemMinSupport.get(cand[0])):
                            print("pruning check condition met")
                            print("list s,", list(s))
                            if list(s) not in frequent_items:
                                print("removing c: " , cand)
                                candidate_gen.remove(cand)
                                break
    return candidate_gen
                

    
    
    

In [50]:
index_k = 1
while len(frequent_items_set[index_k -1]) > 0: 
    if index_k == 1:
#         l = constraints_check(l, must_have, cannot_have)
        #print("l: ",l)
        candidates_set = list(level_2_candidate_generation(l,sdc))
        print("candidates_set: ",candidates_set)
        
        
    else:
#       frequent_items_set[index_k -1] = constraints_check(frequent_items_set[index_k -1], must_have, cannot_have)
        print(" input to 'MScandidate_gen':", f_k)
        candidates_set = MScandidate_gen(f_k,sdc)
        print("candidates_set %s at index %s" %(candidates_set,index_k))
    for t in transactions:
        #print("transaction: ", t)
        index_c = 0    
        
        for c in candidates_set:
            #print("c:",c)
            try:
                c = list(c.split(','))
            except:
                print("c as list: ",c)
            #print("c.split:",c)
            #print(c_len+1)
            if len(c)<index_k+2:
                c.append(0)
                c.append(0)
            #support count of candidate
            #print("c[:2]:",c[:2])
            #print("c[-2]:",c[-2])
            
            if set(c[:-2]) <= set(t):
                c[-2] = int(c[-2]) + 1
            candidates_set[index_c] = ','.join(str(x) for x in c)
            #tail count of candidate
            if set(c[1:-2]) <= set(t):
                c[-1] = int(c[-1]) + 1
            candidates_set[index_c] = ','.join(str(x) for x in c)
            #print("c after tail and suport count: ", c)   
            index_c += 1   
       
            #print(itemMinSupport.get(c[0]))
            #if float(c[2])/len(transactions) >= float(itemMinSupport.get(c[0])):
    print("appended candidates_set: ",candidates_set)
    f_k = []
    for c in candidates_set:
        c= list(c.split(','))
        if float(c[-2])/len(transactions) >= float(itemMinSupport.get(c[0])): 
            f_k.append(c)
    #frequent_items_set.append([f for f in [list(x.split(',')) for x in candidates_set] if float(f[-2])/len(transactions) >= float(itemMinSupport.get(f[0]))])
    
    frequent_items_set.append(f_k)
    print("frequent item set %s: %s " %(index_k,frequent_items_set))    
    #prune the set based on must and cannot be together constraint
    #frequent_items_set = cannot_be_together_check(frequent_items_set,cannot_have)
    f_k = [f[:-2] for f in f_k]       
    index_k += 1
frequent_items_set = cannot_be_together_check(frequent_items_set,cannot_have)
frequent_items_set = must_have_check(frequent_items_set, must_have)
print("Pruned Frequent_item_set: ", frequent_items_set)

candidates_set:  ['10,80', '10,30', '10,40', '10,60', '10,20', '10,70', '80,30', '80,40', '80,20', '80,70', '30,40', '30,60', '30,20', '30,70', '40,60', '40,20', '40,70', '60,20', '60,70', '60,50']
appended candidates_set:  ['10,80,8,18', '10,30,10,30', '10,40,9,26', '10,60,18,46', '10,20,11,30', '10,70,14,41', '80,30,8,30', '80,40,7,26', '80,20,7,30', '80,70,11,41', '30,40,11,26', '30,60,20,46', '30,20,13,30', '30,70,22,41', '40,60,15,46', '40,20,10,30', '40,70,17,41', '60,20,23,30', '60,70,27,41', '60,50,46,58']
frequent item set 1: [[['10'], ['80'], ['30'], ['40'], ['60'], ['50']], [['10', '80', '8', '18'], ['10', '30', '10', '30'], ['10', '40', '9', '26'], ['10', '60', '18', '46'], ['10', '20', '11', '30'], ['10', '70', '14', '41'], ['30', '60', '20', '46'], ['30', '70', '22', '41'], ['60', '70', '27', '41'], ['60', '50', '46', '58']]] 
 input to 'MScandidate_gen': [['10', '80'], ['10', '30'], ['10', '40'], ['10', '60'], ['10', '20'], ['10', '70'], ['30', '60'], ['30', '70'], ['60'

In [51]:
def output():
    output_file = open("result.txt","w") 
    for i in range(len(frequent_items_set)):
        
        if len(frequent_items_set[i]) >0:
            output_file.write("Frequent %s-itemset " %(i+1)+"\n")
            print("Frequent %s-itemset " %(i+1))              
        if(i == 0) and (len(frequent_items_set[i]) == 0):
            print("Total number of itemsets = 0")
            output_file.write("Total number of itemsets = 0")
        
        if i == 0:
            for k in (frequent_items_set[i]):
                count = 0
                for t in transactions:
                    count += t.count(k[0])
                output_file.write("\t%s : %s "%(count,set(k))+"\n") 
                print("\t%s : %s "%(count,set(k)))
        for j in range(len(frequent_items_set[i])):
            if (i > 0) and (len(frequent_items_set[i]) >0):
                output_file.write("\t%s : %s "%(frequent_items_set[i][j][-2],set(frequent_items_set[i][j][:-2]))+"\n") 
                output_file.write("Tail Count = %s" %(frequent_items_set[i][j][-1])+"\n") 
                print("\t%s : %s "%(frequent_items_set[i][j][-2],set(frequent_items_set[i][j][:-2]))+"\n")
                print ("Tail Count = ", frequent_items_set[i][j][-1]+"\n")
        if len(frequent_items_set[i]) >0:
            print("Total number of itemsets = %s" %(len(frequent_items_set[i])))
            output_file.write("Total number of itemsets = %s" %(len(frequent_items_set[i]))+"\n")
    output_file.close()       
        
                

In [52]:
output()

Frequent 1-itemset 
	30 : {'30'} 
	46 : {'60'} 
Total number of itemsets = 2
Frequent 2-itemset 
	10 : {'10', '30'} 

Tail Count =  30

	18 : {'10', '60'} 

Tail Count =  46

	22 : {'30', '70'} 

Tail Count =  41

	27 : {'70', '60'} 

Tail Count =  41

	46 : {'50', '60'} 

Tail Count =  58

Total number of itemsets = 5
Frequent 3-itemset 
	7 : {'10', '30', '70'} 

Tail Count =  22

	9 : {'20', '10', '60'} 

Tail Count =  23

	9 : {'10', '70', '60'} 

Tail Count =  27

	27 : {'70', '50', '60'} 

Tail Count =  37

Total number of itemsets = 4
